# 📝 Personalized Prompt Engineering
### Building Context‑Aware, User‑Specific LLM Interactions (Colab Notebook)

**Author:** Prompt Engineering Course

---
### Learning goals
1. Understand the rationale and psychology behind prompt personalization.
2. Explore common personalization patterns (slot‑filling, persona injection, memory‑based, etc.).
3. Build data structures & retrieval pipelines that adapt prompts to individual users.
4. Evaluate personalization quality & mitigate privacy / bias risks.
5. Practice hands‑on with OpenAI (or compatible) APIs in Colab.

Run every code cell (⌘/Ctrl + Enter) in order. Feel free to modify & explore!

In [ ]:
#@title ⚙️ Install & import dependencies
!pip -q install --upgrade openai faiss-cpu sentence-transformers

import os, json, time, random
from typing import Dict, List
import openai, faiss
from sentence_transformers import SentenceTransformer, util

#@markdown **⬅️ Paste your OpenAI API key and run**
openai.api_key = "sk-..."  # ← replace with your key
MODEL = "gpt-4o-mini"   # or any chat‑completion model

print('Ready!')

## 1️⃣ Why Personalize Prompts?
- **Cognitive fit** – users process information better when language, domain, and examples match their background.
- **Trust & engagement** – personalized tone increases perceived helpfulness.
- **Task efficiency** – fewer clarifications, higher first‑try success.
- **LLM alignment** – providing user data reduces hallucination via richer context.

> *“Personalization turns a generic assistant into **my** assistant.”*

## 2️⃣ User & Context Modelling
A minimal profile can be a Python dict or a row in a database. Key facets:
1. **Demographics** – language, locale, expertise.
2. **Preferences** – tone, output format, depth.
3. **History** – previous questions, documents, feedback.
4. **Persona facts** – name, pronouns, role.

We'll start with an in‑memory toy profile.

In [ ]:
# Toy profile (expand as needed)
user_profile = {
    "id": "u123",
    "name": "Alex",
    "role": "Data Analyst",
    "preferred_tone": "concise",
    "interests": ["sports analytics", "python", "coffee"],
    "history": []
}
print(json.dumps(user_profile, indent=2))

## 3️⃣ Personalization Patterns
### 3.1 Slot‑Filling Templates
Insert variables into a prompt string.
```python
template = "Hey {name}! Here's a {tone} overview of {topic}."
```

In [ ]:
#@title 🔧 Slot‑Filling Demo
def slot_prompt(profile:Dict, topic:str):
    return (
        f"Hey {profile['name']}! "
        f"Here's a {profile['preferred_tone']} overview of {topic}."
    )

prompt = slot_prompt(user_profile, "vector databases")
print(prompt)

### 3.2 Persona/Role Injection
Add a **system** message describing the user's persona so the LLM adapts style & content.

In [ ]:
#@title 🤖 Persona Injection
def chat_completion(profile, user_question):
    messages=[
        {"role":"system","content":(
            f"You are a helpful assistant responding to {profile['name']}, "
            f"a {profile['role']}. Use a {profile['preferred_tone']} tone."
        )},
        {"role":"user","content":user_question}
    ]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

print(chat_completion(user_profile, "Explain FAISS in 2 sentences."))

### 3.3 Memory‑Based Personalization
Retrieve past interactions (or external documents) similar to the current query and prepend them.

In [ ]:
#@title 📚 Simple Memory Store with FAISS
emb_model = SentenceTransformer('all-MiniLM-L6-v2')

# Build vector index of previous Q&A pairs
dimension = emb_model.get_sentence_embedding_dimension()
memory_index = faiss.IndexFlatL2(dimension)
stored_texts: List[str] = []

def add_memory(text:str):
    vec = emb_model.encode([text])
    memory_index.add(vec)
    stored_texts.append(text)

# Add a couple of memories
add_memory('Alex likes quick bullet‑points.')
add_memory('Alex asked about cosine similarity last week.')

def retrieve_memories(query:str, k=2):
    vec = emb_model.encode([query])
    D,I = memory_index.search(vec,k)
    return [stored_texts[i] for i in I[0] if i<len(stored_texts)]

print(retrieve_memories('similarity search'))

In [ ]:
#@title 🧠 Memory‑Aware Chat Helper
def personalized_chat(profile, question):
    memories = '\n'.join(retrieve_memories(question, k=2))
    system_msg = (
        f"You are chatting with {profile['name']} (a {profile['role']}). "
        f"Tone: {profile['preferred_tone']}. Relevant memories:\n{memories}"
    )
    messages=[{"role":"system","content":system_msg},
              {"role":"user","content":question}]
    return openai.chat.completions.create(model=MODEL, messages=messages).choices[0].message.content

print(personalized_chat(user_profile, 'Could you revisit cosine similarity briefly?'))

## 4️⃣ Evaluating Personalization Quality
| Metric | Description |
|---|---|
| Relevance | Does the answer reflect user profile (tone, domain)? |
| Task success | Did the user achieve their goal? |
| Engagement | Click‑through, follow‑up rate. |
| Preference score | Explicit thumbs‑up/down or RLHF. |
| Fairness & bias | No undue stereotyping or exclusion. |

Below is a very lightweight automated check using embedding similarity between the prompt and the user profile.

In [ ]:
#@title ⚖️ Simple Embedding Similarity Scorer
def personalization_score(profile, response):
    profile_text = ' '.join([profile['role']] + profile['interests'])
    vec1, vec2 = emb_model.encode([profile_text, response])
    score = util.cos_sim(vec1, vec2).item()
    return score

resp = personalized_chat(user_profile, 'Recommend a coffee playlist for coding.')
print(resp)
print('Personalization score (0‑1 approx):', round(personalization_score(user_profile, resp), 3))

## 5️⃣ Ethical & Privacy Considerations
- **Data minimization**: store only what you need.
- **Transparency**: tell users how their data is used.
- **Opt‑out**: allow disabling personalization.
- **Bias mitigation**: watch for stereotypes introduced through profiling.
- **Security**: encrypt PII at rest and in transit.

Remember: *With great personalization comes great responsibility.*

## 6️⃣ Hands‑On Exercises
1. **Template Builder** – expand `slot_prompt` to support fallback defaults if profile keys are missing.
2. **Multi‑User Support** – refactor the memory store to index by `profile['id']`.
3. **RAG Personalization** – connect FAISS to an external knowledge base (e.g., Markdown files) and condition answers on both user profile & retrieved docs.
4. **A/B Test Prompt Tone** – compare user satisfaction when `preferred_tone` is obeyed vs ignored.
5. **Privacy Audit** – list every place PII is stored in this notebook and propose mitigations.

## 7️⃣ Further Reading & Resources
- OpenAI docs: *Personalizing ChatGPT* (2024)
- Liu et al., *Persona‑based Dialog Models* (ACL, 2021)
- Google PAIR, *People + AI Guidebook* – Personalization chapter
- Blog: *Dynamic Prompt Engineering with User Embeddings* (2025)
- GitHub: `p-e/prompt‑personalization‑toolkit`

---
🏁 **End of notebook** – experiment, iterate & enjoy personalized prompts!